## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page <font color='red'> For Seller to update:[Title_of_your_product](Provide link to your marketplace listing of your product).</font>
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

## Explain Voice Of Patient (VOP) Pipeline


- **Model**: `en.explain_doc.clinical_vop.pipeline`
- **Model Description**: This pipeline is designed to extract all healthcare-related entities, assign assertion status to the extracted entities, and establish relations between the extracted entities from the clinical documents.


In [1]:
model_package_arn = "<Customer to specify Model package ARN corresponding to their AWS region>"

In [2]:
import base64
import json
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
import boto3
from IPython.display import Image, display
from PIL import Image as ImageEdit
import numpy as np

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/base_serializers.py:28: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 1.22.4)
  import scipy.sparse


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
sagemaker_session = sage.Session()
s3_bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
role = get_execution_role()

sagemaker = boto3.client("sagemaker")
s3_client = sagemaker_session.boto_session.client("s3")
ecr = boto3.client("ecr")
sm_runtime = boto3.client("sagemaker-runtime")

In [9]:
model_name = "en-explain-doc-clinical-vop-pipeline"

real_time_inference_instance_type = "ml.m4.xlarge"
batch_transform_inference_instance_type = "ml.m4.2xlarge"

## 2. Create a deployable model from the model package.

In [10]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, 
    model_package_arn=model_package_arn,
    sagemaker_session=sagemaker_session,
)

### Input Format

To use the model, you need to provide input in one of the following supported formats:

#### JSON Format

Provide input as JSON. We support two variations within this format:

1. **Array of Text Documents**: 
   Use an array containing multiple text documents. Each element represents a separate text document.

   ```json
   {
       "text": [
           "Text document 1",
           "Text document 2",
           ...
       ]
   }

    ```

2. **Single Text Document**:
   Provide a single text document as a string.


   ```json
    {
        "text": "Single text document"
    }
   ```

#### JSON Lines (JSONL) Format

Provide input in JSON Lines format, where each line is a JSON object representing a text document.

```
{"text": "Text document 1"}
{"text": "Text document 2"}
```

## 3. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

### A. Deploy the SageMaker model to an endpoint

In [11]:
# Deploy the model
predictor = model.deploy(
    initial_instance_count=1,
    instance_type=real_time_inference_instance_type, 
    endpoint_name=model_name,
)

--------!

Once endpoint has been created, you would be able to perform real-time inference.

In [12]:
import json
import pandas as pd
import os
import boto3

# Set display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

def process_data_and_invoke_realtime_endpoint(data, content_type, accept):

    content_type_to_format = {'application/json': 'json', 'application/jsonlines': 'jsonl'}
    input_format = content_type_to_format.get(content_type)
    if content_type not in content_type_to_format.keys() or accept not in content_type_to_format.keys():
        raise ValueError("Invalid content_type or accept. It should be either 'application/json' or 'application/jsonlines'.")

    i = 1
    input_dir = f'inputs/real-time/{input_format}'
    output_dir = f'outputs/real-time/{input_format}'
    s3_input_dir = f"{model_name}/validation-input/real-time/{input_format}"
    s3_output_dir = f"{model_name}/validation-output/real-time/{input_format}"

    input_file_name = f'{input_dir}/input{i}.{input_format}'
    output_file_name = f'{output_dir}/{os.path.basename(input_file_name)}.out'

    while os.path.exists(input_file_name) or os.path.exists(output_file_name):
        i += 1
        input_file_name = f'{input_dir}/input{i}.{input_format}'
        output_file_name = f'{output_dir}/{os.path.basename(input_file_name)}.out'

    os.makedirs(os.path.dirname(input_file_name), exist_ok=True)
    os.makedirs(os.path.dirname(output_file_name), exist_ok=True)

    input_data = json.dumps(data) if content_type == 'application/json' else data

    # Write input data to file
    with open(input_file_name, 'w') as f:
        f.write(input_data)

    # Upload input data to S3
    s3_client.put_object(Bucket=s3_bucket, Key=f"{s3_input_dir}/{os.path.basename(input_file_name)}", Body=bytes(input_data.encode('UTF-8')))

    # Invoke the SageMaker endpoint
    response = sm_runtime.invoke_endpoint(
        EndpointName=model_name,
        ContentType=content_type,
        Accept=accept,
        Body=input_data,
    )

    # Read response data
    response_data = json.loads(response["Body"].read().decode("utf-8")) if accept == 'application/json' else response['Body'].read().decode('utf-8')

    # Save response data to file
    with open(output_file_name, 'w') as f_out:
        if accept == 'application/json':
            json.dump(response_data, f_out, indent=4)
        else:
            for item in response_data.split('\n'):
                f_out.write(item + '\n')

    # Upload response data to S3
    output_s3_key = f"{s3_output_dir}/{os.path.basename(output_file_name)}"
    if accept == 'application/json':
        s3_client.put_object(Bucket=s3_bucket, Key=output_s3_key, Body=json.dumps(response_data).encode('UTF-8'))
    else:
        s3_client.put_object(Bucket=s3_bucket, Key=output_s3_key, Body=response_data)

    return response_data

### Initial Setup

In [13]:
docs = [
    "My nose is dry most of the time, when my nose is not dry, I have to clear it of hard mucus – when I do that, it makes my nose throb.",
    "Morning Symptoms:Excessive Mucus build upExcessive coughing 10 minutes after waking upFuzzy sinus headache every morningSore throat every morningBloated stomachExtremely tight chest (Has an inhaler for the asthma)Mid-day Symptoms:Fuzzy headache throughout the dayExtreme fatigue around 3pm every dayEnd-of-day Symptoms:Headache increases to be unbearable.",
]

sample_text = """I had been feeling really tired all the time and was losing weight without even trying. My doctor checked my sugar levels and they came out to be high. So, I have type 2 diabetes. He put me on two medications - I take metformin 500 mg twice a day, and glipizide 5 mg before breakfast and dinner. I also have to watch what I eat and try to exercise more. Now, I also have chronic acid reflux disease or GERD. Now I take daily omeprazole 20 mg to control the heartburn symptoms."""

### JSON

#### Example 1

In [14]:
input_json_data = {"text": sample_text}

data =  process_data_and_invoke_realtime_endpoint(input_json_data, content_type="application/json" , accept="application/json" )
ner_df = pd.DataFrame(data[0]["ner_predictions"])
assertion_df = pd.DataFrame(data[0]["assertion_predictions"])
relation_df = pd.DataFrame(data[0]["relation_predictions"])

In [15]:
ner_df

,ner_chunk,begin,end,ner_label,ner_confidence
0,feeling really tired,11,30,Symptom,0.42706665
1,all the time,32,43,Duration,0.5925
2,losing weight,53,65,Symptom,0.7063
3,doctor,91,96,Employment,0.9753
4,sugar levels,109,120,Test,0.40135002
5,high,146,149,TestResult,0.8967
6,type 2 diabetes,163,177,Disease_Syndrome_Disorder,0.98926663
7,He,180,181,Gender,1.0
8,metformin,218,226,Drug,0.9863
9,500 mg,228,233,Strength,0.9979


In [16]:
assertion_df

,ner_chunk,begin,end,ner_label,assertion,assertion_confidence
0,feeling really tired,11,30,Symptom,Present_Or_Past,0.9999
1,losing weight,53,65,Symptom,Present_Or_Past,0.9983
2,sugar levels,109,120,Test,Present_Or_Past,0.886
3,high,146,149,TestResult,Present_Or_Past,0.5596
4,type 2 diabetes,163,177,Disease_Syndrome_Disorder,Present_Or_Past,0.5282
5,metformin,218,226,Drug,Present_Or_Past,0.958
6,glipizide,252,260,Drug,Present_Or_Past,0.8632
7,exercise,339,346,HealthStatus,Hypothetical_Or_Absent,0.6934
8,chronic acid reflux disease,371,397,Disease_Syndrome_Disorder,Present_Or_Past,1.0
9,GERD,402,405,Disease_Syndrome_Disorder,Present_Or_Past,0.9688


In [17]:
relation_df

,ner_chunk1,ner_chunk1_begin,ner_chunk1_end,ner_label1,ner_chunk2,ner_chunk2_begin,ner_chunk2_end,ner_label2,relations,relation_confidence
0,sugar levels,109,120,Test,high,146,149,TestResult,Test-TestResult,1.0
1,metformin,218,226,Drug,500 mg,228,233,Strength,Drug-Strength,1.0
2,metformin,218,226,Drug,twice a day,235,245,Frequency,Drug-Frequency,1.0
3,metformin,218,226,Drug,glipizide,252,260,Drug,Drug-Drug,1.0
4,metformin,218,226,Drug,5 mg,262,265,Strength,Drug-Strength,1.0
5,500 mg,228,233,Strength,glipizide,252,260,Drug,Strength-Drug,1.0
6,glipizide,252,260,Drug,5 mg,262,265,Strength,Drug-Strength,1.0
7,glipizide,252,260,Drug,before breakfast and dinner,267,293,Frequency,Drug-Frequency,1.0
8,daily,419,423,Frequency,omeprazole,425,434,Drug,Frequency-Drug,1.0
9,omeprazole,425,434,Drug,20 mg,436,440,Strength,Drug-Strength,1.0


### JSON Lines

In [18]:
import json

def create_jsonl(records):
    
    if isinstance(records, str):
        records = [records]
    
    json_records = []

    for text in records:
        record = {
            "text": text
        }
        json_records.append(record)

    json_lines = '\n'.join(json.dumps(record) for record in json_records)

    return json_lines

#### Example 1

In [19]:
input_jsonl_data = create_jsonl(sample_text)
data = process_data_and_invoke_realtime_endpoint(input_jsonl_data, content_type="application/jsonlines" , accept="application/jsonlines" )
print(data)

{"ner_predictions": [{"ner_chunk": "feeling really tired", "begin": 11, "end": 30, "ner_label": "Symptom", "ner_confidence": "0.42706665"}, {"ner_chunk": "all the time", "begin": 32, "end": 43, "ner_label": "Duration", "ner_confidence": "0.5925"}, {"ner_chunk": "losing weight", "begin": 53, "end": 65, "ner_label": "Symptom", "ner_confidence": "0.7063"}, {"ner_chunk": "doctor", "begin": 91, "end": 96, "ner_label": "Employment", "ner_confidence": "0.9753"}, {"ner_chunk": "sugar levels", "begin": 109, "end": 120, "ner_label": "Test", "ner_confidence": "0.40135002"}, {"ner_chunk": "high", "begin": 146, "end": 149, "ner_label": "TestResult", "ner_confidence": "0.8967"}, {"ner_chunk": "type 2 diabetes", "begin": 163, "end": 177, "ner_label": "Disease_Syndrome_Disorder", "ner_confidence": "0.98926663"}, {"ner_chunk": "He", "begin": 180, "end": 181, "ner_label": "Gender", "ner_confidence": "1.0"}, {"ner_chunk": "metformin", "begin": 218, "end": 226, "ner_label": "Drug", "ner_confidence": "0.98

### C. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [20]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 4. Batch inference

In [21]:
import json
import os

input_dir = 'inputs/batch'
json_input_dir = f"{input_dir}/json"
jsonl_input_dir = f"{input_dir}/jsonl"

output_dir = 'outputs/batch'
json_output_dir = f"{output_dir}/json"
jsonl_output_dir = f"{output_dir}/jsonl"

os.makedirs(json_input_dir, exist_ok=True)
os.makedirs(jsonl_input_dir, exist_ok=True)
os.makedirs(json_output_dir, exist_ok=True)
os.makedirs(jsonl_output_dir, exist_ok=True)

validation_json_file_name = "input.json"

validation_jsonl_file_name = "input.jsonl"

validation_input_json_path = f"s3://{s3_bucket}/{model_name}/validation-input/batch/json/"
validation_output_json_path = f"s3://{s3_bucket}/{model_name}/validation-output/batch/json/"

validation_input_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-input/batch/jsonl/"
validation_output_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-output/batch/jsonl/"

def write_and_upload_to_s3(input_data, file_name):
    file_format = os.path.splitext(file_name)[1].lower()
    if file_format == ".json":
        input_data = json.dumps(input_data)

    with open(file_name, "w") as f:
        f.write(input_data)

    s3_client.put_object(
        Bucket=s3_bucket,
        Key=f"{model_name}/validation-input/batch/{file_format[1:]}/{os.path.basename(file_name)}",
        Body=(bytes(input_data.encode("UTF-8"))),
    )

In [22]:
input_jsonl_data = create_jsonl(docs)
input_json_data = {"text": docs}

write_and_upload_to_s3(input_json_data, f"{json_input_dir}/{validation_json_file_name}")

write_and_upload_to_s3(input_jsonl_data, f"{jsonl_input_dir}/{validation_jsonl_file_name}")

### JSON

In [ ]:
# Initialize a SageMaker Transformer object for making predictions
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/json",
    output_path=validation_output_json_path
)

transformer.transform(validation_input_json_path, content_type="application/json")
transformer.wait()

In [24]:
from urllib.parse import urlparse

def process_s3_json_output_and_save(validation_file_name):

    output_file_path = f"{json_output_dir}/{validation_file_name}.out"
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = json.loads(response["Body"].read().decode("utf-8"))
    # Define a dictionary with DataFrame objects and their names
    display(data)
    # Save the data to the output file
    with open(output_file_path, 'w') as f_out:
        json.dump(data, f_out, indent=4)

In [25]:
process_s3_json_output_and_save(validation_json_file_name)

[{'ner_predictions': [{'ner_chunk': 'nose',
    'begin': 3,
    'end': 6,
    'ner_label': 'BodyPart',
    'ner_confidence': '0.5773'},
   {'ner_chunk': 'dry',
    'begin': 11,
    'end': 13,
    'ner_label': 'Symptom',
    'ner_confidence': '0.9457'},
   {'ner_chunk': 'nose',
    'begin': 41,
    'end': 44,
    'ner_label': 'BodyPart',
    'ner_confidence': '0.866'},
   {'ner_chunk': 'dry',
    'begin': 53,
    'end': 55,
    'ner_label': 'Symptom',
    'ner_confidence': '0.9596'},
   {'ner_chunk': 'hard mucus',
    'begin': 80,
    'end': 89,
    'ner_label': 'Symptom',
    'ner_confidence': '0.30905002'},
   {'ner_chunk': 'nose',
    'begin': 121,
    'end': 124,
    'ner_label': 'BodyPart',
    'ner_confidence': '0.9249'}],
  'assertion_predictions': [{'ner_chunk': 'dry',
    'begin': 11,
    'end': 13,
    'ner_label': 'Symptom',
    'assertion': 'Present_Or_Past',
    'assertion_confidence': '0.8839'},
   {'ner_chunk': 'dry',
    'begin': 53,
    'end': 55,
    'ner_label': 'Symp

### JSON Lines

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/jsonlines",
    output_path=validation_output_jsonl_path
)
transformer.transform(validation_input_jsonl_path, content_type="application/jsonlines")
transformer.wait()

In [27]:
from urllib.parse import urlparse

def process_s3_jsonlines_output_and_save(validation_file_name):

    output_file_path = f"{jsonl_output_dir}/{validation_file_name}.out"
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = response["Body"].read().decode("utf-8")
    print(data)

    # Save the data to the output file
    with open(output_file_path, 'w') as f_out:
        for item in data.split('\n'):
            f_out.write(item + '\n')

In [28]:
process_s3_jsonlines_output_and_save(validation_jsonl_file_name)

{"ner_predictions": [{"ner_chunk": "nose", "begin": 3, "end": 6, "ner_label": "BodyPart", "ner_confidence": "0.5773"}, {"ner_chunk": "dry", "begin": 11, "end": 13, "ner_label": "Symptom", "ner_confidence": "0.9457"}, {"ner_chunk": "nose", "begin": 41, "end": 44, "ner_label": "BodyPart", "ner_confidence": "0.866"}, {"ner_chunk": "dry", "begin": 53, "end": 55, "ner_label": "Symptom", "ner_confidence": "0.9596"}, {"ner_chunk": "hard mucus", "begin": 80, "end": 89, "ner_label": "Symptom", "ner_confidence": "0.30905002"}, {"ner_chunk": "nose", "begin": 121, "end": 124, "ner_label": "BodyPart", "ner_confidence": "0.9249"}], "assertion_predictions": [{"ner_chunk": "dry", "begin": 11, "end": 13, "ner_label": "Symptom", "assertion": "Present_Or_Past", "assertion_confidence": "0.8839"}, {"ner_chunk": "dry", "begin": 53, "end": 55, "ner_label": "Symptom", "assertion": "Hypothetical_Or_Absent", "assertion_confidence": "0.9674"}, {"ner_chunk": "hard mucus", "begin": 80, "end": 89, "ner_label": "Sym

In [29]:
model.delete_model()

INFO:sagemaker:Deleting model with name: en-explain-doc-clinical-vop-pipeline-2024-09-02-07-06-52-684


### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

